In [78]:
from random import seed
from random import random
from math import exp

In [79]:
#定义initialize_network,用于构建和初始化神经网约路,n_inputs输入神经元数里,n_hidden隐含层神经元的个数,n_outputs输出神经元的个数。
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    #隐含层,多出一个权值是留给偏置的
    hidden_layer = [{'weights': [random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    
    output_layer = [{'weights': [random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    
    return network


In [80]:
# 定义train_network，根据指定的训练周期训练网络。
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        if epoch % 100 == 0: ## 每隔若干轮才打印输出信息，以便监控训练过程。
            print('>周期=%d, 误差=%.3f' % (epoch, sum_error))

In [81]:
#定义forward_propagate,用于计算神经网络的正向传播,row为输入。
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs=new_inputs # 更新下一层的输入
    return inputs


In [82]:
#定义activate,用于计算神经元的激活值(加权之和)。
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation


In [83]:
#定义激活函数。此处用的是Sigmoid。
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))


In [84]:
# 定义backward_propagate_error,用于反向传传播误差信息,并将纠偏责任存储在神经元中。
def backward_propagate_error(network, expected):
    ## reversed能返回一个反转的迭代器,i是从1开始再到0,所以先到输出层。
    temp = reversed(range(len(network)))
    for i in reversed(range(len(network))):

        layer = network[i]
        errors = list()

        if i!=len(network)-1: # 隐含层
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['responsibility'])
                errors.append(error)
        else: # 输出层
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
         #第一遍先计算输出层误差(也叫"纠偏责任"),第二遍是计算隐含层神经元误差(也叫"纠偏责任")。
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['responsibility'] = errors[j] * transfer_derivative(neuron['output'])


In [85]:
#定义激活函数的导数。
def transfer_derivative(output):
    return output * (1.0 - output)


In [86]:
# 定义update_weights，用于根据误差，更新网络权重。
def update_weights(network, row, l_rate): 
    for i in range(len(network)):
        inputs = row[:-1]  # 0到倒数第一个，不要最后一个，最后一个为预期值
        if i != 0:  # 如果不是输入层,inputs会改变
            # 上一层的输出就是这一层的输入
            inputs = [neuron['output'] for neuron in network[i - 1]]            
        for neuron in network[i]:
            for j in range(len(inputs)):
                # 本身权值加上误差，从而更新
                neuron['weights'][j] += l_rate * neuron['responsibility'] * inputs[j]
            # 最后一个权值是偏置的权值，则默认输入为1
            neuron['weights'][-1] += l_rate * neuron['responsibility']
    ## 测试网络
    test_dataset = [[0, 1, 1],
                [1, 0, 1],
                [1, 1, 0],
                [0, 0, 0]]
    print('3、测试网络:')
    for row in test_dataset:
        prediction = predict(network, row)
        print('预期值=%d, 实际输出值=%d' % (row[-1], prediction))

In [87]:
# 定义predict，用于test。
def predict(network, row):
    outputs = forward_propagate(network, row)
    # 找到最大输出的位置
    return outputs.index(max(outputs))

In [88]:
# 主函数
if __name__ == '__main__':
    # 测试BP网络
    seed(2)

    ## 定义数据
    dataset = [[1, 1, 0],
               [1, 0, 1],
               [0, 1, 1],
               [0, 0, 0]]
    n_inputs = len(dataset[0]) - 1

    n_outputs = len(set([row[-1] for row in dataset]))

    print('1、定义网络并初始化。')
    network = initialize_network(n_inputs, 2, n_outputs)
    print('-- 初始化网络为:')
    print(network)

    ## 打印网络参数
    '''for layer in network:
        print('-- 网络各层参数为:')
        print(layer)'''

    print('2、训练网络:')
    train_network(network, dataset, 0.5, 1000, n_outputs)
    print('   训练结束。')